In [222]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import json

### Check if Chord Diagram is Feasible


In [5]:
df_t = pd.read_excel('trade-in-vehicles.xls')

In [7]:
df_t.columns

Index([u'Trade_in_Vehicle_Category',             u'Trade_in_make',
                  u'Trade_in_Model',             u'Trade_in_Year',
                           u'Count'],
      dtype='object')

In [6]:
df_p = pd.read_excel('new-vehicles.xls')

In [8]:
df_p.columns

Index([u'New_Vehicle_Category',     u'New_Vehicle_Make',
          u'New_Vehicle_Model',     u'New_Vehicle_Year',
                      u'Count'],
      dtype='object')

In [9]:
df_t = df_t.groupby(['Trade_in_make'])['Count'].sum().reset_index()
df_p = df_p.groupby(['New_Vehicle_Make'])['Count'].sum().reset_index()

In [16]:
print 'number unique trade-in makers:', len(df_t['Trade_in_make'].unique())
print 'number unique new-vehicle makers:', len(df_p['New_Vehicle_Make'].unique())

number unique trade-in makers: 60
number unique new-vehicle makers: 35


In [30]:
print df_t.sort('Count', ascending = False).head()
print df_t.sort('Count', ascending = False).head()

   Trade_in_make   Count
19          Ford  195406
13     Chevrolet  118562
15         Dodge   74028
29          Jeep   63368
20           GMC   34486
   Trade_in_make   Count
19          Ford  195406
13     Chevrolet  118562
15         Dodge   74028
29          Jeep   63368
20           GMC   34486


In [14]:
# scrape for automaker by country: https://en.wikipedia.org/wiki/List_of_automobile_manufacturers

35

In [33]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_automobile_manufacturers')

In [37]:
soup = BeautifulSoup(r.content, 'lxml')

In [93]:
auto_maker_dict = dict()
for headline in soup.findAll('h3'):
    if re.search(r'\[edit\]$', headline.text.strip()):
        country = headline.text[0:-6]
        if headline.findNextSibling().text.strip() == 'Current[edit]':
            for status in headline.findNextSiblings('ul', limit=2):
                for maker in status.find_all('a'):
                    auto_maker_dict[maker.text] = country               
        else:
            for maker in headline.find_next('ul').find_all('a'):
                auto_maker_dict[maker.text] = country               


In [108]:
auto_maker_dict['Audi']

u'Germany'

In [171]:
df_am = pd.DataFrame({'maker': auto_maker_dict.keys(),'country': auto_maker_dict.values()})

In [ ]:
# make matrix for chord

In [411]:
df = pd.read_csv('data.txt')

In [104]:
df.columns

Index([u'vendor_id', u'dealer_name', u'address_line1', u'address_line2',
       u'address_line3', u'address_line4', u'city', u'state', u'ZIP',
       u'area_code', u'phone', u'invoice_id', u'invoice_num', u'invoice_date',
       u'sale_date', u'disposal_status', u'disposal_facility_nmvtis_id',
       u'disposal_facility_contact_info', u'sales_type', u'invoice_amount',
       u'trade_in_VIN', u'trade_in_vehicle_category', u'trade_in_make',
       u'trade_in_model', u'trade_in_year', u'trade_in_vehicle_drive_train',
       u'trade_in_mileage', u'trade_in_title_state',
       u'trade_in_registration_state', u'trade_in_registration_start',
       u'trade_in_registration_end', u'trade_in_insurance_start',
       u'trade_in_NMVTIS_flag', u'trade_in_odometer_reading',
       u'new_vehicle_VIN_trunc', u'new_vehicle_category', u'new_vehicle_make',
       u'new_vehicle_model', u'new_vehicle_year', u'new_vehicle_drive_train',
       u'new_vehicle_car_mileage', u'new_vehicle_MSRP'],
      dtype='o

In [277]:
df = df[['trade_in_make','new_vehicle_make']]

In [172]:
df_c = df.merge(df_am, left_on = 'trade_in_make', right_on='maker').merge(df_am, left_on = 'new_vehicle_make', right_on='maker')

In [173]:
df_c.columns

Index([u'trade_in_make', u'new_vehicle_make', u'country_x', u'maker_x',
       u'country_y', u'maker_y'],
      dtype='object')

In [174]:
df_c = df_c[['country_x', 'country_y']]

In [279]:
df_c.columns = ['trade_in','new_vehicle']

In [176]:
df_test = df.merge(df_am, how='left', left_on = 'trade_in_make', right_on='maker')

In [177]:
df_test = df_test.ix[df_test['country'].isnull(),:]

In [178]:
df_test['trade_in_make'].unique()

array(['Eagle', 'AM General', 'American Motors Corporation',
       'Import Trade Services', 'J.K. Motors', 'Sterling',
       'Lambda Control Systems', 'Austin Rover', 'Laforza Automobile Inc',
       'Wallace Environmental', 'ASC Incorporated', 'TVR Engineering Ltd',
       'Excalibur Autos', 'Roush Performance', 'Saturn', 'Federal Coach',
       'PAS, Inc/GMC', 'CCC Engineering', 'Aurora Cars Ltd'], dtype=object)

In [167]:
df_am.ix[df_am['maker'].str.match('For'),:]

,country,maker
34,United States,Ford
83,Canada,Foremost
107,Latvia,Ford-Vairogs


In [168]:
manual_maker = {'Oldsmobile': 'Ford', 'Mercury': 'Ford', 'Merkur': 'Ford', 'Pontiac': 'General Motors', 'Plymouth': 'General Motors'}
manual_country = {'General Motors': 'United States'}

In [169]:
for key, val in manual_maker.iteritems():
    df.ix[df['trade_in_make'] == key, 'trade_in_make'] = val
    df.ix[df['new_vehicle_make'] == key, 'new_vehicle_make'] = val

In [170]:
for key, val in manual_country.iteritems():
    auto_maker_dict[key] = val

In [462]:
df_c['count'] = 1
df_cg = df_c.groupby(['trade_in', 'new_vehicle']).count().reset_index()

In [463]:
df_p = df_cg.pivot(index = 'trade_in', columns = 'new_vehicle', values = 'count')
df_p = df_p.fillna(0)

In [464]:
df_p['France'] = 0
df_p['France'] = 0
df_p['Italy'] = 0
df_p['United Kingdom'] = 0
df_p = df_p.sort_index()
df_p = df_p.reindex_axis(sorted(df_p.columns), axis=1)

In [482]:
with open('matrix.json', 'w') as f:
    f.write(json.dumps(df_p.values.astype(int).tolist()))

In [477]:
df_temp = df_p.reset_index()
df_temp.rename(columns={'trade_in': 'name'}, inplace = True)
df_temp['color'] = range(len(df_temp))
df_temp = df_temp[['name', 'color']]
df_temp.to_json('countries.json',orient='records')

In [445]:
df = pd.read_csv('data.txt')

In [446]:
df = df[['trade_in_make','new_vehicle_make']]
df.columns = ['trade_in','new_vehicle']
df['count'] = 1

df_mg = df.groupby(['trade_in', 'new_vehicle']).count().reset_index()

In [447]:
df_temp = df_mg.groupby('trade_in').sum()
makers_included = df_temp.ix[df_temp['count'] > 1000, :].index

In [448]:
df_p = df_mg.ix[(df_mg['count'] > 10) & (df_mg['trade_in'].isin(makers_included)) & (df_mg['new_vehicle'].isin(makers_included)), :].pivot(index = 'trade_in', columns = 'new_vehicle', values = 'count')
df_p = df_p.fillna(0)


In [449]:
for col in df_p.index.difference(df_p.columns):
    df_p[col] = 0

In [450]:
for row in df_p.columns.difference(df_p.index):
    df_p.loc[row ,:] = 0

In [451]:
df_p = df_p.sort_index()
df_p = df_p.reindex_axis(sorted(df_p.columns), axis=1)

In [452]:
with open('matrix1.json', 'w') as f:
    f.write(json.dumps(df_p.values.astype(int).tolist()))

In [453]:
df_temp = df_p.reset_index()
df_temp.rename(columns={'trade_in': 'name'}, inplace = True)
df_temp['color'] = range(len(df_temp))
df_temp = df_temp[['name', 'color']]
df_temp.to_json('countries1.json',orient='records')

In [483]:
df_p

new_vehicle,France,Germany,Italy,Japan,South Korea,Sweden,United Kingdom,United States
trade_in,,,,,,,,
France,0,1,0,15,2,1,0,2
Germany,0,907,0,6803,1107,43,0,1424
Italy,0,3,0,17,5,0,0,6
Japan,0,2261,0,58633,8498,130,0,10880
South Korea,0,52,0,1739,1170,4,0,740
Sweden,0,183,0,1861,302,42,0,370
United Kingdom,0,243,0,1780,350,44,0,626
United States,0,10973,0,248344,66324,752,0,241050


In [454]:
df = pd.read_csv('data.txt')

In [455]:
df = df[['trade_in_vehicle_category', 'new_vehicle_category']]
df.columns = ['trade_in','new_vehicle']

In [456]:
df['count'] = 1
df_ctg = df.groupby(['trade_in', 'new_vehicle']).count().reset_index()

In [457]:
df_p = df_ctg.pivot(index = 'trade_in', columns = 'new_vehicle', values = 'count')
df_p = df_p.fillna(0)


In [458]:
for col in df_p.index.difference(df_p.columns):
    df_p[col] = 0
for row in df_p.columns.difference(df_p.index):
    df_p.loc[row ,:] = 0

In [459]:
df_p = df_p.sort_index()
df_p = df_p.reindex_axis(sorted(df_p.columns), axis=1)

In [460]:
with open('matrix2.json', 'w') as f:
    f.write(json.dumps(df_p.values.astype(int).tolist()))

In [461]:
df_temp = df_p.reset_index()
df_temp.rename(columns={'trade_in': 'name'}, inplace = True)
df_temp['color'] = range(len(df_temp))
df_temp = df_temp[['name', 'color']]
df_temp.to_json('countries2.json',orient='records')